In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse('./appleヘルスケア190706.xml')

In [3]:
root = tree.getroot()

In [4]:
root[10000].attrib['type']

'HKQuantityTypeIdentifierHeartRate'

In [5]:
import pandas as pd

BPSystolic = []
BPS_datetime = []

BPDiastolic = []
BPD_datetime = []

HRate = []
HRate_datetime = []

for child in root:
    data = child.attrib

    try:
        if data['type'] == 'HKQuantityTypeIdentifierBloodPressureSystolic':
            BPS_datetime.append(data['startDate'])
            BPSystolic.append(data['value'])
        elif data['type'] == 'HKQuantityTypeIdentifierBloodPressureDiastolic':
            BPD_datetime.append(data['startDate'])
            BPDiastolic.append(data['value'])
        elif data['type'] == 'HKQuantityTypeIdentifierHeartRate':
            HRate_datetime.append(data['startDate'])
            HRate.append(data['value'])
    except:
        pass

In [6]:
df_BPS = pd.DataFrame({
    'BPS_datetime': BPS_datetime,
    'BPS': BPSystolic
})
df_BPS['BPS_datetime'] = pd.to_datetime(df_BPS['BPS_datetime'])

df_BPD = pd.DataFrame({
    'BPD_datetime': BPD_datetime,
    'BPD': BPDiastolic
})
df_BPD['BPD_datetime'] = pd.to_datetime(df_BPD['BPD_datetime'])

df_HRate = pd.DataFrame({
    'HRate_datetime': HRate_datetime,
    'HRate': HRate
})
df_HRate['HRate_datetime'] = pd.to_datetime(df_HRate['HRate_datetime'])
df_HRate = df_HRate.sort_values(by='HRate_datetime')

In [7]:
df_BPS.head()

,BPS_datetime,BPS
0,2018-09-28 07:47:00+09:00,136
1,2018-09-29 09:29:00+09:00,132
2,2018-09-30 08:50:00+09:00,135
3,2018-10-01 07:53:00+09:00,129
4,2018-10-02 07:41:00+09:00,140


In [8]:
df_HRate.head()

,HRate_datetime,HRate
0,2018-09-30 08:50:00+09:00,64
1,2018-10-01 07:53:00+09:00,86
2,2018-10-02 07:42:00+09:00,85
3,2018-10-03 07:59:00+09:00,93
4,2018-10-04 07:40:00+09:00,93


In [9]:
df_BP = pd.merge_asof(df_BPS, df_BPD, left_on='BPS_datetime', right_on='BPD_datetime', tolerance=pd.Timedelta('5min'),
                     direction='nearest').drop(columns=['BPD_datetime'])
df_BP.head()

,BPS_datetime,BPS,BPD
0,2018-09-28 07:47:00+09:00,136,74
1,2018-09-29 09:29:00+09:00,132,85
2,2018-09-30 08:50:00+09:00,135,83
3,2018-10-01 07:53:00+09:00,129,86
4,2018-10-02 07:41:00+09:00,140,86


In [10]:
df_BP_HR = pd.merge_asof(df_BP, df_HRate, left_on='BPS_datetime', right_on='HRate_datetime', tolerance=pd.Timedelta('240min'),
                     direction='nearest').drop(columns=['HRate_datetime'])
df_BP_HR

,BPS_datetime,BPS,BPD,HRate
0,2018-09-28 07:47:00+09:00,136,74,NaN
1,2018-09-29 09:29:00+09:00,132,85,NaN
2,2018-09-30 08:50:00+09:00,135,83,64
3,2018-10-01 07:53:00+09:00,129,86,86
4,2018-10-02 07:41:00+09:00,140,86,85
5,2018-10-03 07:59:00+09:00,126,88,93
6,2018-10-04 07:40:00+09:00,129,82,93
7,2018-10-05 07:28:00+09:00,127,84,79
8,2018-10-06 09:30:00+09:00,133,86,70
9,2018-10-07 08:48:00+09:00,131,87,64


In [11]:
df_BP_HR['BPS_datetime'] = df_BP_HR['BPS_datetime'].astype(str)
f = lambda x:x[:-6]
df_BP_HR['BPS_datetime'] = df_BP_HR['BPS_datetime'].map(f)

In [12]:
df_BP_HR.head()

,BPS_datetime,BPS,BPD,HRate
0,2018-09-28 07:47:00,136,74,NaN
1,2018-09-29 09:29:00,132,85,NaN
2,2018-09-30 08:50:00,135,83,64
3,2018-10-01 07:53:00,129,86,86
4,2018-10-02 07:41:00,140,86,85


In [13]:
df_BP_HR.to_csv("./healthcare.csv")

QuantityTypeIdentifier	表示名
HKQuantityTypeIdentifierBodyMassIndex	ボディマス指数（BMI)
HKQuantityTypeIdentifierBodyFatPercentage	体脂肪率
HKQuantityTypeIdentifierHeight	身長
HKQuantityTypeIdentifierBodyMass	体重
HKQuantityTypeIdentifierLeanBodyMass	除脂肪体重
HKQuantityTypeIdentifierStepCount	歩数
HKQuantityTypeIdentifierDistanceWalkingRunning	ウォーキング＋ランニング
HKQuantityTypeIdentifierDistanceCycling	自転車の走行距離
HKQuantityTypeIdentifierBasalEnergyBurned	基礎代謝量
HKQuantityTypeIdentifierActiveEnergyBurned	アクティブカロリー
HKQuantityTypeIdentifierFlightsClimbed	上った回数
HKQuantityTypeIdentifierNikeFuel	NikeFuel
HKQuantityTypeIdentifierHeartRate	心拍数
HKQuantityTypeIdentifierBodyTemperature	体温
HKQuantityTypeIdentifierBloodPressureSystolic	最高血圧
HKQuantityTypeIdentifierBloodPressureDiastolic	最低血圧
HKQuantityTypeIdentifierRespiratoryRate	呼吸数
HKQuantityTypeIdentifierOxygenSaturation	酸素飽和度
HKQuantityTypeIdentifierPeripheralPerfusionIndex	微小循環指標
HKQuantityTypeIdentifierBloodGlucose	血糖値
HKQuantityTypeIdentifierNumberOfTimesFallen	倒れた回数
HKQuantityTypeIdentifierElectrodermalActivity	皮膚電位
HKQuantityTypeIdentifierInhalerUsage	吸入器の使用状況
HKQuantityTypeIdentifierBloodAlcoholContent	血中アルコール濃度
HKQuantityTypeIdentifierForcedVitalCapacity	努力肺活量
HKQuantityTypeIdentifierForcedExpiratoryVolume1	努力呼気肺活量（1秒）
HKQuantityTypeIdentifierPeakExpiratoryFlowRate	最大呼気速度
HKQuantityTypeIdentifierDietaryFatTotal	総脂肪
HKQuantityTypeIdentifierDietaryFatPolyunsaturated	多価不飽和脂肪酸
HKQuantityTypeIdentifierDietaryFatMonounsaturated	一価飽和脂肪酸
HKQuantityTypeIdentifierDietaryFatSaturated	飽和脂肪酸
HKQuantityTypeIdentifierDietaryCholesterol	食物中のコレステロール
HKQuantityTypeIdentifierDietarySodium	ナトリウム
HKQuantityTypeIdentifierDietaryCarbohydrates	炭水化物
HKQuantityTypeIdentifierDietaryFiber	食物繊維
HKQuantityTypeIdentifierDietarySugar	糖分
HKQuantityTypeIdentifierDietaryEnergyConsumed	摂取カロリー
HKQuantityTypeIdentifierDietaryProtein	たんぱく質
HKQuantityTypeIdentifierDietaryVitaminA	ビタミンA
HKQuantityTypeIdentifierDietaryVitaminB6	ビタミンB6
HKQuantityTypeIdentifierDietaryVitaminB12	ビタミンB12
HKQuantityTypeIdentifierDietaryVitaminC	ビタミンC
HKQuantityTypeIdentifierDietaryVitaminD	ビタミンD
HKQuantityTypeIdentifierDietaryVitaminE	ビタミンE
HKQuantityTypeIdentifierDietaryVitaminK	ビタミンK
HKQuantityTypeIdentifierDietaryCalcium	カルシウム
HKQuantityTypeIdentifierDietaryIron	鉄
HKQuantityTypeIdentifierDietaryThiamin	チアミン
HKQuantityTypeIdentifierDietaryRiboflavin	ビタミンB2
HKQuantityTypeIdentifierDietaryNiacin	ナイアシン
HKQuantityTypeIdentifierDietaryFolate	葉酸
HKQuantityTypeIdentifierDietaryBiotin	ビオチン
HKQuantityTypeIdentifierDietaryPantothenicAcid	パントテン酸
HKQuantityTypeIdentifierDietaryPhosphorus	リン
HKQuantityTypeIdentifierDietaryIodine	ヨウ素
HKQuantityTypeIdentifierDietaryMagnesium	マグネシウム
HKQuantityTypeIdentifierDietaryZinc	亜鉛
HKQuantityTypeIdentifierDietarySelenium	セレン
HKQuantityTypeIdentifierDietaryCopper	銅
HKQuantityTypeIdentifierDietaryManganese	マンガン
HKQuantityTypeIdentifierDietaryChromium	クロム
HKQuantityTypeIdentifierDietaryMolybdenum	モリブデン
HKQuantityTypeIdentifierDietaryChloride	塩化物
HKQuantityTypeIdentifierDietaryPotassium	カリウム
HKQuantityTypeIdentifierDietaryCaffeine	カフェイン
HKCategoryTypeIdentifierSleepAnalysis	-
HKCharacteristicTypeIdentifierBiologicalSex	-
HKCharacteristicTypeIdentifierBloodType	-
HKCharacteristicTypeIdentifierDateOfBirth	-
HKCorrelationTypeIdentifierBloodPressure	-
HKCorrelationTypeIdentifierFood	-
HKWorkoutTypeIdentifier	-